In [1]:
from math import sqrt
import copy
import  traceback
import shutil

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image
from tqdm import tqdm

In [2]:
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices

In [3]:
lower_b, upper_b = -45, 205

## 1.阳性数据

In [ ]:
def generate_image(input_folder):
    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

generate_image('/nfs3-p1/zsxm/dataset/2021-9-8/')
generate_image('/nfs3-p1/zsxm/dataset/2021-9-13/')
generate_image('/nfs3-p1/zsxm/dataset/2021-9-19/')

In [ ]:
def move_together_for_detect(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-8/', '/nfs3-p1/zsxm/dataset/9_for_detect/')
move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-13/', '/nfs3-p1/zsxm/dataset/9_for_detect/')
move_together_for_detect('/nfs3-p1/zsxm/dataset/2021-9-19/', '/nfs3-p1/zsxm/dataset/9_for_detect/')

In [ ]:
def directly_move_to_image_label(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        image_path = os.path.join(dst_path, name, 'images')
        if os.path.exists(image_path):
            print(f"\tremove {image_path}")
            shutil.rmtree(image_path)

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), image_path)
        except:
            traceback.print_exc()
            
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')

In [ ]:
def move_ori_image_to_sigle_folder(input_path, dst_path):
    for patient in os.listdir(input_path):
        print(f'****Processing {patient}****')
        if not os.path.exists(os.path.join(dst_path, patient)):
            os.mkdir(os.path.join(dst_path, patient))
        image_path = os.path.join(dst_path, patient, 'images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for img in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, img)):
                shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
    
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-8/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-13/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-19/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')

In [ ]:
def move_detect_result_to_single_folder(input_path, output_path)
    for patient in os.listdir(input_path):
        print(f'Processing {patient}')
        image_path = os.path.join(output_path, patient, 'pred_images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for file in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, file)):
                shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
            elif os.path.isdir(os.path.join(input_path, patient, file)):
                if os.path.exists(os.path.join(input_path, patient, file)):
                    shutil.rmtree(os.path.join(input_path, patient, file))
                shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
                
move_detect_result_to_single_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-8_pred/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
move_detect_result_to_single_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-13_pred/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive')
move_detect_result_to_single_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-19_pred/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new')

In [11]:
input_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new'
error_patient_list = []

def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.25 < y < 0.75, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.25 < corr2[2] < 0.75, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = max(w, h), max(w, h)
    return int(width*(x-w/2)), int(height*(y-h/2)), int(width*(x+w/2)+1), int(height*(y+h/2)+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.lower().split('-')
        for i, s in enumerate(pl):
            if i!=0 and s.startswith('im'):
                pl[i] = s.replace('im', '')
        crop_flag = True
        for i, s in enumerate(pl):
            if s != 's' and s != 'j':
                continue
            start, end = int(pl[i+1])-1, int(pl[i+2])
            for j in range(start, end):
                img = Image.open(os.path.join(image_path, f'{patient}_{j:04d}.png'))
                img = np.array(img)
                try:
                    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
                except:
                    traceback.print_exc()
                    crop_flag = False
                else:#if crop_flag:
                    crop = img[y1:y2, x1:x2]
                    crop = Image.fromarray(crop)
                    crop.save(os.path.join(crop_path, f'{patient}_{s}_{j:04d}.png'))
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)

epl1 = []
epl2 = []
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive', epl1)
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new', epl2)

******Processing baoyin-S-Im9-22-J-Im9-68******


Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 9, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：baoyin-S-Im9-22-J-Im9-68_0019.txt


******Processing caijilu-J-Im41-87******
******Processing caimeiju-J-Im21-81******
******Processing caiweiguang-J-Im35-152******
******Processing caiying-J-Im34-52******
******Processing chenaijun-S-Im25-38-J-Im25-85******
******Processing chenazhao-S-Im17-30******
******Processing chenbo-S-Im24-45-J-Im24-97******
******Processing chenfujun-J-Im43-100******
******Processing chengdazhong-J-Im24-38******
******Processing chenggang-J-Im18-81******
******Processing chenglongdeng-S-Im21-33-J-Im21-78******
******Processing chengshizheng-J-Im30-145******
******Processing chenguorong-J-Im18-77******
******Processing chenjian-J-Im14-81******
******Processing chenjinfa-J-Im34-105******
******Processing chenjuli-J-Im18-49******
******Processing chenjun-J-Im33-137******
******Processing chenlibo-S-Im39-57******
******Processing chenlili-S-Im18-24-J-Im18-74******
******Processing chenmimao-S-Im32-40******
******Processing chenmin-J-Im26-67******
******Processing chenping-J-Im23-88******
******Proce

Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/fanghao-J-Im23-89/labels/fanghao-J-Im23-89_0075.txt'
Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/fanghao-J-Im23-89/labels/fanghao-J-Im23-89_0085.txt'


******Processing fangtianming-S-Im15-30-J-Im15-65******


...

Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 9, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhengchong-S-16-31-J-16-87_0018.txt


******Processing zhongchuankun-J-17-69******


Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/zhongchuankun-J-17-69/labels/zhongchuankun-J-17-69_0041.txt'
Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/zhongchuankun-J-17-69/labels/zhongchuankun-J-17-69_0061.txt'


******Processing zhoujianglong-S-17-31******
******Processing zhuxinjian-J-20-23******


Traceback (most recent call last):
  File "<ipython-input-11-01e2a0fa49b2>", line 48, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-11-01e2a0fa49b2>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/zhuxinjian-J-20-23/labels/zhuxinjian-J-20-23_0019.txt'


In [12]:
print(len(epl1))
print(epl1)
print(len(epl2))
print(epl2)

66
['baoyin-S-Im9-22-J-Im9-68', 'fanghao-J-Im23-89', 'fangtianming-S-Im15-30-J-Im15-65', 'fanxinan-S-Im16-36-J-Im16-30', 'gongyongfa-S-Im18-34-J-Im18-87', 'guanxujun-S-Im25-35-J-Im25-87', 'guquankang-J-Im18-69', 'heguoyu-J-Im36-141', 'heyuexiang-J-Im18-76', 'huajin-J-Im30-67', 'huangxihong-J-Im27-136', 'laiguizhen-J-Im38-121', 'laixuexiang-J-Im33-148', 'lanshaomei-S-Im20-31-J-Im20-79', 'leitufa-S-Im30-60-J-Im30-117', 'leixiaoying-S-Im18-26-J-18-71', 'lianzhaozhen-S-Im23-30-J-Im23-84', 'ligang-S-Im33-36', 'lingshounv-S-Im18-28-J-Im18-76', 'liwenjun-J-Im33-132', 'liyehua-S-Im30-47-J-Im30-131', 'louyidong-S-Im38-58-J-Im38-133', 'luzhiping-S-Im25-35-J-Im25-54', 'lvxiangshen-J-Im32-139', 'maliwei-S-Im30-48-J-Im30-101', 'penggaofeng-S-Im28-36', 'pengzhengjiang-J-Im38-160', 'ruweiping-S-Im52-63-J-Im52-94', 'shenjie-S-Im39-59-J-Im39-85', 'tangabiao-S-Im41-49-J-Im41-138', 'tongjinglin-J-Im36-146', 'wangchenrong-S-Im18-24-J-Im18-38', 'wangjiangwei-J-Im37-143', 'wangyaotang-J-Im31-100', 'wangzhux

In [13]:
def move_to_classify(input_root, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for patient in os.listdir(input_root):
        if os.path.isfile(os.path.join(input_root, patient)):
            continue
        print(f'****Processing {patient}****')
        for file in os.listdir(os.path.join(input_root, patient, 'crops')):
            shutil.copy(os.path.join(input_root, patient, 'crops', file), output_path)

move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/1')
move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/positive_new/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/1')

****Processing chenaijun-S-Im25-38-J-Im25-85****
****Processing chenazhao-S-Im17-30****
****Processing chengshizheng-J-Im30-145****
****Processing chenjuli-J-Im18-49****
****Processing chenlibo-S-Im39-57****
****Processing chenmin-J-Im26-67****
****Processing chenqiusheng-J-Im34-143****
****Processing chujiancheng-J-Im45-122****
****Processing dingzhiqun-S-Im21-29****
****Processing fangtianming-S-Im15-30-J-Im15-65****
****Processing gaosheng-J-Im19-67****
****Processing gaoxiangqin-J-Im24-75****
****Processing guanxujun-S-Im25-35-J-Im25-87****
****Processing guoheying-J-Im32-127****
****Processing hanchun-J-Im20-86****
****Processing heguoyu-J-Im36-141****
****Processing hexingwen-S-Im25-35-J-Im25-93****
****Processing hongjinjun-S-Im24-28-J-Im24-83****
****Processing hongzhimin-S-Im30-39-J-30-144****
****Processing huajin-J-Im30-67****
****Processing huangqiang-J-Im42-147****
****Processing huangxiansheng-J-Im25-59****
****Processing huangxihong-J-Im27-136****
****Processing huangyon

****Processing wangjiangwei-J-Im37-143****
****Processing wangweibo-J-Im36-149****
****Processing wangxinchun-J-Im18-81****
****Processing wangyaotang-J-Im31-100****
****Processing wangyefu-J-Im42-112****
****Processing wangyonghui-S-Im18-27-J-Im18-84****
****Processing wangzhuxi-J-Im25-144****
****Processing weiguanxin-J-Im43-116****
****Processing weinaichao-J-Im59-162****
****Processing weizhiqing-J-Im33-45****
****Processing wenyongguo-J-Im23-121****
****Processing wujufen-J-Im31-135****
****Processing wuwanglong-J-Im31-125****
****Processing wuxiangyang-S-Im18-25****
****Processing wuyingbo-J-Im36-98****
****Processing wuyueming-J-Im36-138****
****Processing xiaminsong-S-Im40-51-J-Im40-139****
****Processing xiayunqing-J-Im33-141****
****Processing xiechangyou-S-Im22-31-J-Im22-84****
****Processing xuanhuili-J-Im30-135****
****Processing xuhong-S-Im23-38-J-Im23-40****
****Processing xushangfa-J-Im36-141****
****Processing xuyanfang-J-Im20-80****
****Processing xuyaofeng-J-Im24-119

## 2.阴性数据

In [4]:
def generate_image(input_folder):
    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, f'images_{lower_b}_{upper_b}')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, lower_b, upper_b)
                img = ((img-lower_b)/(upper_b-lower_b)*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

generate_image('/nfs3-p2/zsxm/dataset/2021-9-17-negative/')

****Processing baixiaoxu-22-31-52-147****
****Processing baojinjia-17-21-35-83****
****Processing baoyanzu-21-25-42-93****
****Processing bianyinqiao-31-38-59-137****
****Processing bianzhaorong-17-22-38-92****
****Processing caicunyuan-12-16-30-86****
****Processing caijingnan1-41-50-73-184****
****Processing caiqishu-34-43-70-176****
****Processing caiyouzhe-14-18-32-83****
****Processing caoyuxia-16-21-35-87****
****Processing chenchuanmu-44-52-91-184****
****Processing chenfugui-36-46-76-149****
****Processing chengzhiqiang-33-41-67-159****
****Processing chenhefa-17-22-38-81****
****Processing chenjian-26-30-44-89****
****Processing chenjianhua-14-17-32-80****
****Processing chenjianjun-25-34-65-175****
****Processing chenjianping-15-22-41-91****
****Processing chenjingjing-38-47-67-184****
****Processing chenjinmei-12-17-34-80****
****Processing chenpanyang-21-31-71-161****
****Processing chenqiuying-13-17-30-78****
****Processing chensaimu-34-44-78-166****
****Processing chenxiu

In [ ]:
def move_together_for_detect(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)
    root_name = input_folder.split('/')[-1] if input_folder.split('/')[-1] != '' else input_folder.split('/')[-2]
    dst_path = os.path.join(dst_path, root_name)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()

move_together_for_detect('/nfs3-p2/zsxm/dataset/2021-9-17-negative', '/nfs3-p1/zsxm/dataset/9_for_detect/')

In [10]:
def directly_move_to_image_label(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        image_path = os.path.join(dst_path, name, 'images')
        if os.path.exists(image_path):
            print(f"\tremove {image_path}")
            shutil.rmtree(image_path)

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', f'images_{lower_b}_{upper_b}'), image_path)
        except:
            traceback.print_exc()
            
directly_move_to_image_label('/nfs3-p1/zsxm/dataset/2021-9-17-negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

****Processing baixiaoxu-22-31-52-147****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baixiaoxu-22-31-52-147/images
****Processing baojinjia-17-21-35-83****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baojinjia-17-21-35-83/images
****Processing baoyanzu-21-25-42-93****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baoyanzu-21-25-42-93/images
****Processing bianyinqiao-31-38-59-137****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/bianyinqiao-31-38-59-137/images
****Processing bianzhaorong-17-22-38-92****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/bianzhaorong-17-22-38-92/images
****Processing caicunyuan-12-16-30-86****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caicunyuan-12-16-30-86/images
****Processing caijingnan1-41-50-73-184****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/caijingnan1-41-50-73-184/images
****Processing caiqishu-34-43-70-176****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_

****Processing lihonggang-13-18-33-88****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/lihonggang-13-18-33-88/images
****Processing linaizhen-23-31-58-144****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/linaizhen-23-31-58-144/images
****Processing linshangjin-15-19-36-81****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/linshangjin-15-19-36-81/images
****Processing lirongchang-13-17-27-86****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/lirongchang-13-17-27-86/images
****Processing lishankui-17-25-56-142****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/lishankui-17-25-56-142/images
****Processing liuxiaolin-39-49-81-176****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/liuxiaolin-39-49-81-176/images
****Processing lixiebing-17-20-37-87****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/lixiebing-17-20-37-87/images
****Processing liyingsheng-13-17-34-87****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_

****Processing wuzongtang-29-36-76-141****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/wuzongtang-29-36-76-141/images
****Processing xiabao-33-42-78-196****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiabao-33-42-78-196/images
****Processing xiangbaiquan-27-37-71-156****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiangbaiquan-27-37-71-156/images
****Processing xiaofuxin-25-33-64-153****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiaofuxin-25-33-64-153/images
****Processing xiasuifu-12-18-30-84****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiasuifu-12-18-30-84/images
****Processing xiyongmin-25-35-62-162****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xiyongmin-25-35-62-162/images
****Processing xucaifeng-21-25-52-88****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/xucaifeng-21-25-52-88/images
****Processing xujianhua-16-20-39-87****
	remove /nfs3-p1/zsxm/dataset/aorta_ct_img_label/nega

In [ ]:
def move_ori_image_to_sigle_folder(input_path, dst_path):
    for patient in os.listdir(input_path):
        print(f'****Processing {patient}****')
        if not os.path.exists(os.path.join(dst_path, patient)):
            os.mkdir(os.path.join(dst_path, patient))
        image_path = os.path.join(dst_path, patient, 'images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for img in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, img)):
                shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
    
move_ori_image_to_sigle_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-17_negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

In [ ]:
def move_detect_result_to_single_folder(input_path, output_path)
    for patient in os.listdir(input_path):
        print(f'Processing {patient}')
        image_path = os.path.join(output_path, patient, 'pred_images')
        if os.path.exists(image_path):
            shutil.rmtree(image_path)
        os.mkdir(image_path)
        for file in os.listdir(os.path.join(input_path, patient)):
            if os.path.isfile(os.path.join(input_path, patient, file)):
                shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
            elif os.path.isdir(os.path.join(input_path, patient, file)):
                if os.path.exists(os.path.join(input_path, patient, file)):
                    shutil.rmtree(os.path.join(input_path, patient, file))
                shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))
        os.rmdir(os.path.join(input_path, patient))
    os.rmdir(input_path)
                
move_detect_result_to_single_folder('/nfs3-p1/zsxm/dataset/9_for_detect/2021-9-17_negative/', '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative')

In [14]:
def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.25 < y < 0.75, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.25 < corr2[2] < 0.75, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = max(w, h), max(w, h)
    return int(width*(x-w/2)), int(height*(y-h/2)), int(width*(x+w/2)+1), int(height*(y+h/2)+1)

def crop_images(input_path, error_patient_list):
    for patient in sorted(os.listdir(input_path)):
        print(f'******Processing {patient}******')
        image_path = os.path.join(input_path, patient, 'images')
        label_path = os.path.join(input_path, patient, 'labels')
        crop_path = os.path.join(input_path, patient, 'crops')
        if os.path.exists(crop_path):
            shutil.rmtree(crop_path)
        os.mkdir(crop_path)
        pl = patient.split('-')
        aorta_start, branch_start, branch_end, aorta_end = int(pl[1])-1, int(pl[2])-1, int(pl[3])-1, int(pl[4])-1
        crop_flag = True
        offset = branch_end - branch_start
        start, end = branch_start + int(0.1*offset), branch_end - int(0.2*offset)
        for i in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 's')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_s_{i:04d}.png'))
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png'))
        offset = aorta_end - branch_end
        start, end = branch_end + int(0.1*offset), aorta_end - int(0.2*offset)
        for i in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{i:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_j_{i:04d}.png')) 
        if not crop_flag:
            #print('Delete crop_path')
            #shutil.rmtree(crop_path)
            error_patient_list.append(patient)
            
epl1 = []
crop_images('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative', epl1)

******Processing baixiaoxu-22-31-52-147******


Traceback (most recent call last):
  File "<ipython-input-14-8684d21a6010>", line 64, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-14-8684d21a6010>", line 2, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baixiaoxu-22-31-52-147/labels/baixiaoxu-22-31-52-147_0069.txt'


******Processing baojinjia-17-21-35-83******


Traceback (most recent call last):
  File "<ipython-input-14-8684d21a6010>", line 64, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-14-8684d21a6010>", line 2, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/baojinjia-17-21-35-83/labels/baojinjia-17-21-35-83_0056.txt'


******Processing baoyanzu-21-25-42-93******
******Processing bianyinqiao-31-38-59-137******


...

******Processing zhudamin-33-41-70-156******
******Processing zhujianping-23-29-57-149******
******Processing zhumeizheng-34-42-69-167******


Traceback (most recent call last):
  File "<ipython-input-14-8684d21a6010>", line 64, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-14-8684d21a6010>", line 2, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhumeizheng-34-42-69-167/labels/zhumeizheng-34-42-69-167_0146.txt'


******Processing zhuxiaoqin-22-29-50-130******
******Processing zhuxinnan-18-26-49-129******


Traceback (most recent call last):
  File "<ipython-input-14-8684d21a6010>", line 64, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-14-8684d21a6010>", line 2, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhuxinnan-18-26-49-129/labels/zhuxinnan-18-26-49-129_0107.txt'
Traceback (most recent call last):
  File "<ipython-input-14-8684d21a6010>", line 64, in crop_images
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{i:04d}.txt'), 'j')
  File "<ipython-input-14-8684d21a6010>", line 2, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/zhuxinnan-18-26-49-129/labels/zhuxinnan-18-26-49-129_0108.txt'
Traceback (most recent c

******Processing zoudeling-11-15-30-84******


In [15]:
print(len(epl1))
print(epl1)

107
['baixiaoxu-22-31-52-147', 'baojinjia-17-21-35-83', 'bianyinqiao-31-38-59-137', 'bianzhaorong-17-22-38-92', 'caijingnan1-41-50-73-184', 'caiqishu-34-43-70-176', 'chenchuanmu-44-52-91-184', 'chenfugui-36-46-76-149', 'chenjian-26-30-44-89', 'chenjianhua-14-17-32-80', 'chenjianjun-25-34-65-175', 'chenjianping-15-22-41-91', 'chenjingjing-38-47-67-184', 'chenjinmei-12-17-34-80', 'chenyanli-20-23-34-80', 'chenyinfen-18-22-38-81', 'cheshengying-25-34-64-139', 'denghuibin-15-19-35-84', 'dingbinghai-20-24-45-95', 'dingguanyun-19-24-40-94', 'dinglianhua-19-28-61-143', 'dujinlian-17-20-36-82', 'fangchengfu-26-39-76-182', 'fangrurong-16-25-55-132', 'fudianqin-15-19-33-78', 'geqingqing-33-41-68-169', 'guxinmei-13-18-39-79', 'heguansheng-15-20-43-92', 'hexiena-18-22-38-82', 'huangfujuan-38-47-80-178', 'huangguixian-24-28-45-101', 'huangjuzhen-13-17-29-77', 'huangweigen-24-32-62-138', 'huangxiang-22-26-42-84', 'hukaipin-29-39-73-159', 'jiangyuxiang-15-19-37-83', 'jinaixiang-34-41-63-149', 'jinang

In [16]:
def move_to_classify(input_root, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    for patient in os.listdir(input_root):
        if os.path.isfile(os.path.join(input_root, patient)):
            continue
        print(f'****Processing {patient}****')
        for file in os.listdir(os.path.join(input_root, patient, 'crops')):
            shutil.copy(os.path.join(input_root, patient, 'crops', file), output_path)
            
move_to_classify('/nfs3-p1/zsxm/dataset/aorta_ct_img_label/negative/', f'/nfs3-p1/zsxm/dataset/aorta_classify_ct_{lower_b}_{upper_b}/0/')

****Processing baixiaoxu-22-31-52-147****
****Processing baojinjia-17-21-35-83****
****Processing baoyanzu-21-25-42-93****
****Processing bianyinqiao-31-38-59-137****
****Processing bianzhaorong-17-22-38-92****
****Processing caicunyuan-12-16-30-86****
****Processing caijingnan1-41-50-73-184****
****Processing caiqishu-34-43-70-176****
****Processing caiyouzhe-14-18-32-83****
****Processing caoyuxia-16-21-35-87****
****Processing chenchuanmu-44-52-91-184****
****Processing chenfugui-36-46-76-149****
****Processing chengzhiqiang-33-41-67-159****
****Processing chenhefa-17-22-38-81****
****Processing chenjian-26-30-44-89****
****Processing chenjianhua-14-17-32-80****
****Processing chenjianjun-25-34-65-175****
****Processing chenjianping-15-22-41-91****
****Processing chenjingjing-38-47-67-184****
****Processing chenjinmei-12-17-34-80****
****Processing chenpanyang-21-31-71-161****
****Processing chenqiuying-13-17-30-78****
****Processing chensaimu-34-44-78-166****
****Processing chenxiu